In [50]:
class Rule:
    def __init__(self,left,right,probability):
        self.left=left
        self.right=right
        self.probability=probability

    def __str__(self):
        return f"{self.left}->{self.right} {self.probability}"
        

In [112]:
import pickle
rules=pickle.load(open("rules.data",'rb'))
for i in rules:
    print(i)
z=len(rules)

S->['NP', 'VP'] 1.0
PP->['P', 'NP'] 1.0
VP->['V', 'NP'] 0.7
VP->['VP', 'PP'] 0.3
P->['WITH'] 1.0
V->['SAW'] 1.0
NP->['NP', 'PP'] 0.4
NP->['ASTRONOMERS'] 0.1
NP->['EARS'] 0.18
NP->['SAW'] 0.04
NP->['STARS'] 0.18
NP->['TELESCOPE'] 0.1


In [90]:
words=[]
n=int(input("Enter number of words"))
for i in range(n):
    words.append(input(f"enter {i+1} the word").upper())
print(words)

Enter number of words 5
enter 1 the word astronomers
enter 2 the word saw
enter 3 the word stars
enter 4 the word with
enter 5 the word telescope


['ASTRONOMERS', 'SAW', 'STARS', 'WITH', 'TELESCOPE']


In [104]:
def findRuleWithWord(wrd):
    a=[]
    for rule in rules:
        if rule.right == [wrd]:  # match only terminal productions
            a.append(deepcopy(rule))  # deepcopy is critical
    return a


In [100]:
def findCombinations(x,y):
    comb=[]
    for i in range(x,y):
        comb.append(((x,i),(i+1,y)))
    return comb

In [106]:
from copy import deepcopy
matrix=[[[] for i in range(len(words))]for j in range(len(words))]
for i in range(n):
    for j in range(n):
        if i+j < len(words):
            x=j
            y=i+j
            if x==y:
                matrix[x][y]=findRuleWithWord(words[x])
            else:
                comb=findCombinations(x,y)
                data=[]
                for k in comb:
                    f=matrix[k[0][0]][k[0][1]]
                    g=matrix[k[1][0]][k[1][1]]
                    for p in f:
                        for q in g:
                            for r in rules:
                                if r.right==[p.left,q.left]:
                                    a=deepcopy(r)
                                    a.probability = p.probability*q.probability*r.probability
                                    data.append(a)
                matrix[x][y]=data
for row in matrix:
    print([[str(rule) for rule in cell] for cell in row])

[["NP->['ASTRONOMERS'] 0.1"], [], ["S->['NP', 'VP'] 0.0126"], [], ["S->['NP', 'VP'] 0.000504", "S->['NP', 'VP'] 0.000378"]]
[[], ["V->['SAW'] 1.0", "NP->['SAW'] 0.04"], ["VP->['V', 'NP'] 0.126"], [], ["VP->['V', 'NP'] 0.005039999999999999", "VP->['VP', 'PP'] 0.00378"]]
[[], [], ["NP->['STARS'] 0.18"], [], ["NP->['NP', 'PP'] 0.0072"]]
[[], [], [], ["P->['WITH'] 1.0"], ["PP->['P', 'NP'] 0.1"]]
[[], [], [], [], ["NP->['TELESCOPE'] 0.1"]]


[["NP->['ASTRONOMERS'] 0.1"], [], ["S->['NP', 'VP'] 0.0126"], [], ["S->['NP', 'VP'] 0.000504", "S->['NP', 'VP'] 0.000378"]]
[[], ["V->['SAW'] 1.0", "NP->['SAW'] 0.04"], ["VP->['V', 'NP'] 0.126"], [], ["VP->['V', 'NP'] 0.005039999999999999", "VP->['VP', 'PP'] 0.00378"]]
[[], [], ["NP->['STARS'] 0.18"], [], ["NP->['NP', 'PP'] 0.0072"]]
[[], [], [], ["P->['WITH'] 1.0"], ["PP->['P', 'NP'] 0.1"]]
[[], [], [], [], ["NP->['TELESCOPE'] 0.1"]]


In [114]:
from copy import deepcopy

# Define Rule class
class Rule:
    def __init__(self, left, right, probability):
        self.left = left
        self.right = right
        self.probability = probability

    def __str__(self):
        return f"{self.left}->[{', '.join(self.right)}] {self.probability}"

# Define rules (same as image)
rules = [
    Rule('S', ['NP', 'VP'], 1.0),
    Rule('PP', ['P', 'NP'], 1.0),
    Rule('VP', ['V', 'NP'], 0.7),
    Rule('VP', ['VP', 'PP'], 0.3),
    Rule('P', ['WITH'], 1.0),
    Rule('V', ['SAW'], 1.0),
    Rule('NP', ['ASTRONOMERS'], 0.1),
    Rule('NP', ['EARS'], 0.18),
    Rule('NP', ['SAW'], 0.04),
    Rule('NP', ['STARS'], 0.18),
    Rule('NP', ['TELESCOPES'], 0.1),
]

# Input sentence
words = ['ASTRONOMERS', 'SAW', 'STARS', 'WITH', 'EARS']
n = len(words)

# Function to find rules that produce a terminal word
def findRuleWithWord(word):
    return [deepcopy(rule) for rule in rules if rule.right == [word]]

# Generate combinations of splits
def findCombinations(x, y):
    return [((x, i), (i + 1, y)) for i in range(x, y)]

# Initialize matrix
matrix = [[[] for _ in range(n)] for _ in range(n)]

# CYK Algorithm with inside probabilities
for i in range(n):
    for j in range(n):
        if i + j < n:
            x = j
            y = i + j
            if x == y:
                matrix[x][y] = findRuleWithWord(words[x])
            else:
                combinations = findCombinations(x, y)
                entries = []
                for ((x1, y1), (x2, y2)) in combinations:
                    left_rules = matrix[x1][y1]
                    right_rules = matrix[x2][y2]
                    for p in left_rules:
                        for q in right_rules:
                            for rule in rules:
                                if rule.right == [p.left, q.left]:
                                    new_rule = deepcopy(rule)
                                    new_rule.probability = rule.probability * p.probability * q.probability
                                    entries.append(new_rule)
                matrix[x][y] = entries

# Output in the expected format
print("\nFinal CYK Matrix (Inside Probabilities):\n")
for row in matrix:
    formatted_row = []
    for cell in row:
        if not cell:
            formatted_row.append("[]")
        else:
            rules_str = ", ".join([str(rule) for rule in cell])
            formatted_row.append(f"[{rules_str}]")
    print(formatted_row)


Final CYK Matrix (Inside Probabilities):

['[NP->[ASTRONOMERS] 0.1]', '[]', '[S->[NP, VP] 0.0126]', '[]', '[S->[NP, VP] 0.0006804000000000001]']
['[]', '[V->[SAW] 1.0, NP->[SAW] 0.04]', '[VP->[V, NP] 0.126]', '[]', '[VP->[VP, PP] 0.006804]']
['[]', '[]', '[NP->[STARS] 0.18]', '[]', '[]']
['[]', '[]', '[]', '[P->[WITH] 1.0]', '[PP->[P, NP] 0.18]']
['[]', '[]', '[]', '[]', '[NP->[EARS] 0.18]']


In [31]:
from nltk import PCFG,ViterbiParser
import pandas as pd
import numpy as np
grammar = PCFG.fromstring("""
S -> NN VP [0.50] | VP NP [0.50]
NP -> NN PP [0.70] | PP NN [0.30]
VP -> VB NP [0.30] | VB NN [0.35] | VP NP [0.20] | VB PP [0.15]
PP -> P VP [0.50] | P NN [0.50]
P -> 'with' [0.30] | 'after' [0.25] | 'to' [0.15] | 'without' [0.15] | 'before' [0.10] | 'into' [0.05]
VB -> 'play' [0.20] | 'watch' [0.20] | 'enjoy' [0.20] | 'like' [0.15] | 'listen' [0.10] | 'enjoy' [0.15]
NN -> 'children' [0.15] | 'cricket' [0.15] | 'friends' [0.20] | 'students' [0.10] | 'football' [0.15] | 'painting' [0.25]
""")

vitParser=ViterbiParser(grammar)

sentence=['students', 'like', 'painting']#input("Enter the sentence").strip().split()
# print(sentence)
parsers=list(vitParser.parse(sentence))
# print(parsers)
parseData=[(str(tree),tree.prob()) for tree in parsers]
# print(parseData)
df=pd.DataFrame(parseData,columns=['Parse Tree',"Probability"])
meanProb=np.mean(df['Probability']) if not df.empty else 0
print(meanProb)


from nltk import InsideChartParser
parser=InsideChartParser(grammar)
tokens=["students","like","painting"]
for tree in vitParser.parse(tokens):
    tree.pretty_print()
    print(tree.prob())

0.00065625
          S               
    ______|____            
   |           VP         
   |       ____|_____      
   NN     VB         NN   
   |      |          |     
students like     painting

0.00065625


In [39]:
import numpy as np
import pandas as pd
import nltk
from nltk import PCFG, ViterbiParser

# grammar = PCFG.fromstring("""
# S -> NP VP [0.9] 
# S -> VP [0.1] 
# NP -> Det N [0.8]
# NP -> Det N PP [0.2]
# VP -> V NP [0.7] 
# VP -> V PP [0.3]
# Det -> 'the' [1.0]
# N -> 'cat' [0.6]
# N -> 'dog' [0.4]
# V -> 'chases' [0.7]
# V -> 'sees' [0.3]
# PP -> P NP [1.0]
# P -> 'in' [1.0]
# """)

grammar = PCFG.fromstring("""
S -> NP VP [1.0]
VP -> V NP [0.7] | VP PP [0.3]
PP -> P NP [1.0]
P -> 'with' [1.0]
V -> 'saw' [1.0]
NP -> NP PP [0.4] | 'astronomers' [0.1] | 'ears' [0.18] | 'saw' [0.04] | 'stars' [0.18] | 'telescopes' [0.1]
""")

def main():
    pcfg_grammar = grammar
    viterbi_parser = ViterbiParser(pcfg_grammar)
    
    sentence = input("Enter a sentence to parse: ").strip().split()
    parses = list(viterbi_parser.parse(sentence))
    
    parse_data = [(str(tree), tree.prob()) for tree in parses]
    df = pd.DataFrame(parse_data, columns=['Parse Tree', 'Probability'])
    
    print("\nParse Results using Viterbi Parsing:")
    print(df)
    
    mean_prob = np.mean(df['Probability']) if not df.empty else 0
    print("Mean Probability of Parses:", mean_prob)

if __name__ == "__main__":
    main()

from nltk import InsideChartParser

parser = InsideChartParser(grammar)
sentence=['astronomers','saw','stars','with','ears']
for tree in parser.parse(sentence):
    tree.pretty_print()
    print("PROBABILITY: ",tree.prob())

Enter a sentence to parse:  astronomers saw stars with ears



Parse Results using Viterbi Parsing:
                                          Parse Tree  Probability
0  (S\n  (NP astronomers)\n  (VP (V saw) (NP (NP ...     0.000907
Mean Probability of Parses: 0.0009071999999999999
                  S                    
      ____________|____                 
     |                 VP              
     |        _________|___             
     |       |             NP          
     |       |     ________|____        
     |       |    |             PP     
     |       |    |         ____|___    
     NP      V    NP       P        NP 
     |       |    |        |        |   
astronomers saw stars     with     ears

PROBABILITY:  0.0009071999999999999
                      S                    
      ________________|____                 
     |                     VP              
     |            _________|________        
     |           VP                 PP     
     |        ___|____          ____|___    
     NP      V        NP       